# Semi_defect_finder
반도체 공정/계측 데이터 기반 **결함 여부(불량/정상)** 이진 분류 모델 구축 노트북


[Phase 1] 전처리 파이프라인
VarianceThreshold → 상관관계 제거 → ANOVA top-K

[Phase 2] 데이터 전략별 비교  (핵심 실험: Recall 값 비교)
XGBoost, LightGBM, RF, LR 모델들로
"Normal, Undersampled, SMOTE, PCA + 위 3개" 기법들로 적용 후 비교분석

[Phase 3] 이상치 탐지 (별도 프레임)
EE / IF / LOF → Recall 비교

[Phase 4] Top 3 모델 GridSearchCV 튜닝

[Phase 5] VotingClassifier 앙상블
Phase 2~4에서 Recall 상위 모델 조합





In [ ]:
# =========================================
# 1) Imports (preprocess & modeling)
# =========================================
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

from dataclasses import dataclass
from typing import List, Optional

from sklearn.model_selection import StratifiedKFold, GroupKFold, TimeSeriesSplit, RandomizedSearchCV, train_test_split
from sklearn.metrics import (
    average_precision_score, precision_recall_curve, roc_auc_score,
    f1_score, precision_score, recall_score, confusion_matrix,
    ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay
)
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report
from sklearn.feature_selection import RFE

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.feature_selection import VarianceThreshold

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.experimental import enable_iterative_imputer  # noqa: F401
from sklearn.impute import IterativeImputer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


In [ ]:
# 김태훈
import os
os.chdir("/home/tae-hun-kim/바탕화면/AI모델개발/Semiconductor_Quality_Prospect")
print("cwd:", os.getcwd())
print("exists:", os.path.exists("SECOM/uci-secom.csv"))

In [ ]:
# #최정빈
# import os
# os.chdir(r"C:/Users/0726b/Semiconductor_Quality_Prospect")  
# print("cwd:", os.getcwd())
# print("exists:", os.path.exists("SECOM/uci-secom.csv"))

In [ ]:
# =========================================
# 2) Load data & basic checks
# =========================================

from pathlib import Path
import pandas as pd

# 프로젝트 루트(.git) 탐색
root = Path.cwd()
while root != root.parent and not (root / ".git").exists():
    root = root.parent

csv_path = root / "SECOM" / "uci-secom.csv"
print("Resolved csv_path:", csv_path)

df = pd.read_csv(csv_path)
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

target_col = "Defect"
tool_col = "Tool_Type"

print("Target col:", target_col)
print("Tool_Type col:", tool_col)

print("Missing ratio (top 10):")
print(df.isna().mean().sort_values(ascending=False).head(16))

df.head()


In [ ]:
# =========================================
# 2-1) Drop Time column & correlation heatmap
# =========================================

df = df.drop(columns=["Time"], errors="ignore")
print("Dropped Time:", "Time" in df.columns)

# Numeric-only correlation
corr = df.select_dtypes(include=["number"]).corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Feature Correlation (Time column removed)")
plt.tight_layout()
plt.show()

df.head()


### Time 제거 → 전부 NaN/분산 0 컬럼 제거 → 숫자형 상관계수 계산 → 히트맵으로 시각화.

In [ ]:
# df 기준
df = df.copy()

# 1) 전부 NaN 컬럼
all_nan_cols = df.columns[df.isna().all()].tolist()

# 2) 분산 0 컬럼 (값이 모두 동일)
zero_var_cols = df.columns[df.nunique(dropna=True) <= 1].tolist()

print("all_nan:", len(all_nan_cols))
print("zero_var:", len(zero_var_cols))

# 제거
drop_cols = sorted(set(all_nan_cols + zero_var_cols))
df = df.drop(columns=drop_cols)

# 다시 히트맵
corr = df.select_dtypes(include=["number"]).corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Feature Correlation (cleaned)")
plt.tight_layout()
plt.show()


In [ ]:
df.info()

In [ ]:
# 절댓값 상관계수 0.95 이상인 컬럼 제거
threshold = 0.95

# 숫자형만 상관계수
corr = df.select_dtypes(include=["number"]).corr().abs()

# 상삼각행렬만 사용 (중복 제거)
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

# 임계치 초과 컬럼 선택
to_drop = [col for col in upper.columns if any(upper[col] >= threshold)]

print("Drop count:", len(to_drop))

# 필요하면 목록 확인
print(to_drop)

df = df.drop(columns=to_drop)

df.info()
df.head()

# 제거 후 히트맵
corr_after = df.select_dtypes(include=["number"]).corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_after, cmap="coolwarm", center=0)
plt.title("Feature Correlation (after dropping high-corr columns)")
plt.tight_layout()
plt.show()

In [ ]:
# Pass/Fail을 0/1로 변환 (-1: Pass -> 0, 1: Fail -> 1)
df["Pass/Fail"] = df["Pass/Fail"].map({-1: 0, 1: 1})

df.head(20)

In [ ]:

# 1) 컬럼별 결측률 상위 30개
missing_rate = df.isna().mean().sort_values(ascending=False)
print(missing_rate.head(30))

plt.figure(figsize=(8, 6))
missing_rate.head(30).plot(kind="barh")
plt.title("Top 30 Missing Rates")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# 2) 행별 결측률 분포
row_missing = df.isna().mean(axis=1)

plt.figure(figsize=(6, 4))
sns.histplot(row_missing, bins=30, kde=True)
plt.title("Row-wise Missing Ratio")
plt.tight_layout()
plt.show()

# 3) 결측 패턴 히트맵 (샘플 200행)
sample = df.sample(min(200, len(df)), random_state=42)
plt.figure(figsize=(12, 6))
sns.heatmap(sample.isna(), cbar=False)
plt.title("Missing Pattern (sample)")
plt.tight_layout()
plt.show()

# 4) 타깃별 결측률 비교 (Pass/Fail 컬럼이 있을 때)
if "Pass/Fail" in df.columns:
    missing_by_target = df.groupby("Pass/Fail").apply(lambda x: x.isna().mean())
    print(missing_by_target.head())


In [ ]:
# 결측률 0.5 이상 컬럼 제거
threshold = 0.5
missing_rate = df.isna().mean()
drop_cols = missing_rate[missing_rate >= threshold].index.tolist()

print("Drop count:", len(drop_cols))
print(drop_cols)

df = df.drop(columns=drop_cols)

# 확인
df.info()


In [ ]:
# 1) 컬럼별 결측률 상위 30개
missing_rate = df.isna().mean().sort_values(ascending=False)
print(missing_rate.head(30))

plt.figure(figsize=(8, 6))
missing_rate.head(30).plot(kind="barh")
plt.title("Top 30 Missing Rates")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# 2) 행별 결측률 분포
row_missing = df.isna().mean(axis=1)

plt.figure(figsize=(6, 4))
sns.histplot(row_missing, bins=30, kde=True)
plt.title("Row-wise Missing Ratio")
plt.tight_layout()
plt.show()

# 3) 결측 패턴 히트맵 (샘플 200행)
sample = df.sample(min(200, len(df)), random_state=42)
plt.figure(figsize=(12, 6))
sns.heatmap(sample.isna(), cbar=False)
plt.title("Missing Pattern (sample)")
plt.tight_layout()
plt.show()

# 4) 타깃별 결측률 비교 (Pass/Fail 컬럼이 있을 때)
if "Pass/Fail" in df.columns:
    missing_by_target = df.groupby("Pass/Fail").apply(lambda x: x.isna().mean())
    print(missing_by_target.head())


In [ ]:
target = "Pass/Fail"
X = df.drop(columns=[target])

from sklearn.impute import KNNImputer

# KNN Imputer 적용
imputer = KNNImputer(n_neighbors=5, weights="distance")
X_imputed = imputer.fit_transform(X)

# 다시 DataFrame으로
X_imputed = pd.DataFrame(X_imputed, columns=X.columns)

# KNN으로 채운 결과를 df로 합치기
df = df.copy()
df[X.columns] = X_imputed  # X에 해당하는 컬럼만

# 3) 결측 패턴 히트맵 (샘플 200행)
sample = df.sample(min(200, len(df)), random_state=42)
plt.figure(figsize=(12, 6))
sns.heatmap(sample.isna(), cbar=False)
plt.title("Missing Pattern (after KNN Impute)")
plt.tight_layout()
plt.show()



# Visualization

In [ ]:
# 1) 컬럼 타입 분리
num_cols = df.select_dtypes(include=["number"]).columns.tolist()
cat_cols = df.select_dtypes(exclude=["number"]).columns.tolist()

print("Numeric cols:", len(num_cols))
print("Categorical cols:", len(cat_cols))

# 2) 숫자형: 분산 큰 Top 10 (분포/박스플롯 추천)
num_var = df[num_cols].var().sort_values(ascending=False)
print("\nTop variance numeric cols:")
print(num_var.head(10))

# 3) 숫자형: 왜도 큰 Top 10 (로그변환 전/후 비교 추천)
num_skew = df[num_cols].skew().sort_values(ascending=False)
print("\nTop skew numeric cols:")
print(num_skew.head(10))

# 4) 결측률 높은 컬럼 Top 10 (결측 패턴/타깃 비교 추천)
missing_rate = df.isna().mean().sort_values(ascending=False)
print("\nTop missing rate cols:")
print(missing_rate.head(10))

# 5) 상관 높은 컬럼 쌍 Top 10 (산점도 추천)
corr = df[num_cols].corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
high_corr_pairs = (
    upper.stack()
         .sort_values(ascending=False)
         .head(10)
)
print("\nTop correlated pairs:")
print(high_corr_pairs)

# 6) 범주형: 값 개수 많은 Top 10 (막대그래프/파이 추천)
if cat_cols:
    cat_card = df[cat_cols].nunique().sort_values(ascending=False)
    print("\nTop cardinality categorical cols:")
    print(cat_card.head(10))


In [ ]:
# =========================================
# 4) Target Distribution (Pass/Fail Ratio) - Pastel Style
# =========================================
counts = df['Pass/Fail'].value_counts().sort_index()
labels = ['Pass (0)', 'Fail (1)']
colors = ['#6EC5E9', '#FF8A5B']  # 파스텔 블루/코랄

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar
axes[0].bar(labels, counts.values, color=colors, edgecolor='white', linewidth=1.2)
for i, v in enumerate(counts.values):
    axes[0].text(i, v + 20, f'{v}', ha='center', fontweight='bold')
axes[0].set_ylabel('Sample Count')
axes[0].set_title('Pass/Fail Distribution')

# Pie
axes[1].pie(
    counts.values,
    labels=labels,
    colors=colors,
    autopct='%1.1f%%',
    startangle=90,
    wedgeprops={'edgecolor': 'white', 'linewidth': 1.2},
    textprops={'fontsize': 12}
)
axes[1].set_title('Pass/Fail Ratio')

plt.tight_layout()
plt.show()

print(f'Pass: {counts.iloc[0]} ({counts.iloc[0]/len(df)*100:.1f}%)')
print(f'Fail: {counts.iloc[1]} ({counts.iloc[1]/len(df)*100:.1f}%)')
print(f'Imbalance ratio: 1:{counts.iloc[0]//counts.iloc[1]}')


In [ ]:
X = df.drop(columns=['Pass/Fail'])
y = df['Pass/Fail']

imputer = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)


In [ ]:
# PCA + t-SNE 한 줄(1x2)로 출력
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
pca_df['Target'] = y.values

# t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_scaled)
tsne_df = pd.DataFrame(data=X_tsne, columns=['tSNE1', 'tSNE2'])
tsne_df['Target'] = y.values

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sns.scatterplot(data=pca_df, x='PC1', y='PC2', hue='Target',
                alpha=0.6, palette={-1: 'blue', 1: 'red'}, ax=axes[0])
axes[0].set_title('PCA: 2D Projection of Sensor Data')

sns.scatterplot(data=tsne_df, x='tSNE1', y='tSNE2', hue='Target',
                alpha=0.6, palette={-1: 'blue', 1: 'red'}, ax=axes[1])
axes[1].set_title('t-SNE: 2D Projection of Sensor Data')

plt.tight_layout()
plt.show()


In [ ]:
# RandomForestClassifier 최적화된 피처 중요도 시각화

imputer = SimpleImputer(strategy='median') # 중앙값(Median) 대치
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

selector = VarianceThreshold(threshold=0)
X_clean = pd.DataFrame(selector.fit_transform(X_imputed), columns=X_imputed.columns[selector.get_support()])

rf_best = RandomForestClassifier(
    n_estimators=200, 
    max_depth=10, 
    class_weight='balanced', 
    random_state=42, 
    n_jobs=-1
)

rf_best.fit(X_clean, y)

importances = pd.Series(rf_best.feature_importances_, index=X_clean.columns)
top_features = importances.nlargest(10)

plt.figure(figsize=(12, 8))
sns.barplot(x=top_features.values, y=top_features.index.astype(str), palette='viridis')

plt.title('Feature Importance (Optimized: Median Impute + Balanced Weight + Depth Limit)')
plt.xlabel('Importance Score')
plt.ylabel('Sensor Feature ID')
plt.grid(axis='x', alpha=0.3)
plt.show()

# 상위 10개 출력
print("=== Top 10 Critical Sensors ===")
print(top_features.head(10))

#

In [ ]:
sns.set_style("whitegrid")

target = "Pass/Fail"
top10 = ["59","103","33","510","130","247","129","205","64","21"]
palette_pf = {0: "#6EC5E9", 1: "#FF8A5B", "0": "#6EC5E9", "1": "#FF8A5B"}

# =========================
# 1) 박스플롯 (표준화 + 더 잘 보이게)
# =========================
df_std = df.copy()
df_std[top10] = (df_std[top10] - df_std[top10].mean()) / df_std[top10].std()

df_melt = df_std.melt(id_vars=target, value_vars=top10, var_name="Feature", value_name="Value")

plt.figure(figsize=(12, 5))
sns.boxplot(
    data=df_melt,
    x="Feature",
    y="Value",
    hue=target,
    palette=palette_pf,
    linewidth=1.2,
    fliersize=0,          # 이상치 점 숨김
    whis=1.5
)

ymin, ymax = df_melt["Value"].quantile([0.005, 0.995])
plt.ylim(-5, 5)



plt.title("Top 10 Features by Separation (Standardized, trimmed)")
plt.tight_layout()
plt.show()


# =========================
# 2) 히스토그램 (범례 그래프 밖으로)
# =========================
g0 = df[df[target].astype(str) == "0"]
g1 = df[df[target].astype(str) == "1"]

fig, axes = plt.subplots(2, 5, figsize=(16, 6))
for ax, col in zip(axes.flatten(), top10):
    sns.histplot(g0[col], color="#6EC5E9", label="Pass(0)", kde=True,
                 ax=ax, stat="density", alpha=0.55)
    sns.histplot(g1[col], color="#FF8A5B", label="Fail(1)", kde=True,
                 ax=ax, stat="density", alpha=0.55)
    ax.set_title(str(col), fontsize=10)
    ax.grid(alpha=0.3)

handles, labels = axes[0,0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper left",
           bbox_to_anchor=(0.001, 1.1), frameon=False)

plt.tight_layout(rect=[0, 0, 0.88, 1])  # 오른쪽 여백 확보
plt.show()

# 통계적으로 유의미한 차이 확인 상관계수를 보던지 어떤 컬럼이 중요한지 분석 필요 기준이 없어 일반적으로 상관계수로 함

 # Train / Test data 분리


In [ ]:
target = "Pass/Fail"
X = df.drop(columns=[target])
y = df[target].astype(int)

# 1) 먼저 분리
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

# 2) Train으로만 피처 선택
k = 20
pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("anova", SelectKBest(score_func=f_classif, k=k))
])
pipe.fit(X_train, y_train)

selected_cols = X_train.columns[pipe.named_steps["anova"].get_support()]

# 3) Train/Valid/Test 모두 동일 컬럼만 유지
X_train = X_train[selected_cols]
X_valid = X_valid[selected_cols]
X_test  = X_test[selected_cols]

print("Selected features:", selected_cols.tolist())
print("Train:", X_train.shape)
print("Valid:", X_valid.shape)
print("Test :", X_test.shape)


In [ ]:
# =========================================
# Undersampling 
# =========================================
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=42)
# X_train, y_train = rus.fit_resample(X_train, y_train)

# =========================================
# Oversampling (SMOTE / Borderline-SMOTE)
# =========================================
# 아래 둘 중 하나만 주석 해제해서 사용

# --- SMOTE ---
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state=42)
# X_train, y_train = sm.fit_resample(X_train, y_train)

# --- Borderline-SMOTE ---
# from imblearn.over_sampling import BorderlineSMOTE
# bsm = BorderlineSMOTE(random_state=42, kind='borderline-1')
# X_train, y_train = bsm.fit_resample(X_train, y_train)

print("After Sampling:", X_train.shape, y_train.value_counts().to_dict())

# MODELING

In [ ]:
models = {
    "Logistic": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(max_iter=2000, class_weight="balanced"))
    ]),
    "RandomForest": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", RandomForestClassifier(
            n_estimators=300, max_depth=12, random_state=42,
            n_jobs=-1, class_weight="balanced"
        ))
    ]),
    "HistGB": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", HistGradientBoostingClassifier(
            max_depth=8, learning_rate=0.05, random_state=42
        ))
    ]),
    "XGBoost": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", XGBClassifier(
            n_estimators=300, max_depth=8, learning_rate=0.05,
            random_state=42, n_jobs=-1, eval_metric="logloss"
        ))
    ]),
    "LightGBM": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", LGBMClassifier(
            n_estimators=300, max_depth=8, learning_rate=0.05,
            random_state=42, n_jobs=-1, verbose=-1
        ))
    ])
}

In [ ]:

def eval_model(name, model, X_tr, y_tr, X_va, y_va):
    model.fit(X_tr, y_tr)
    proba = model.predict_proba(X_va)[:, 1]
    pred = (proba >= 0.5).astype(int)
    return {
        "model": name,
        "ROC-AUC": roc_auc_score(y_va, proba),
        "PR-AUC": average_precision_score(y_va, proba),
        "Precision": precision_score(y_va, pred, zero_division=0),
        "Recall": recall_score(y_va, pred, zero_division=0),
        "F1": f1_score(y_va, pred, zero_division=0)
    }

results = []
for name, model in models.items():
    results.append(eval_model(name, model, X_train, y_train, X_valid, y_valid))

pd.DataFrame(results).sort_values("PR-AUC", ascending=False)


In [ ]:

def find_best_threshold(model, X_tr, y_tr, X_va, y_va):
    model.fit(X_tr, y_tr)
    proba = model.predict_proba(X_va)[:, 1]

    thresholds = np.linspace(0.01, 0.99, 99)
    best_f1 = (0, 0, 0)   # (thr, f1, recall)
    best_rec = (0, 0, 0)  # (thr, recall, precision)

    for thr in thresholds:
        pred = (proba >= thr).astype(int)
        f1 = f1_score(y_va, pred, zero_division=0)
        rec = recall_score(y_va, pred, zero_division=0)
        prec = precision_score(y_va, pred, zero_division=0)

        if f1 > best_f1[1]:
            best_f1 = (thr, f1, rec)
        if rec > best_rec[1]:
            best_rec = (thr, rec, prec)

    return best_f1, best_rec

# 5개 모델 전체 검토
for name, model in models.items():
    best_f1, best_rec = find_best_threshold(model, X_train, y_train, X_valid, y_valid)
    print(f"\n{name}")
    print(f"  [Best F1]    thr={best_f1[0]:.2f}, F1={best_f1[1]:.3f}, Recall={best_f1[2]:.3f}")
    print(f"  [Best Recall] thr={best_rec[0]:.2f}, Recall={best_rec[1]:.3f}, Precision={best_rec[2]:.3f}")
    proba = model.predict_proba(X_valid)[:, 1]
    print(f"  PR-AUC: {average_precision_score(y_valid, proba):.3f}")


### 모델별 PR-AUC 성능 비교 결과

| 모델 (Algorithm) | Normal | Undersampling | SMOTE | Borderline-SMOTE |
| :--- | :---: | :---: | :---: | :---: |
| **Logistic Regression** | **0.279** | 0.220 | **0.236** | 0.215 |
| **Random Forest** | 0.234 | 0.226 | 0.183 | 0.216 |
| **HistGB** | 0.186 | 0.212 | 0.203 | 0.171 |
| **XGBoost** | 0.221 | 0.172 | 0.159 | **0.252** |
| **LightGBM** | 0.182 | **0.256** | 0.166 | 0.218 |

---

#### 💡 결과 분석 요약
1. **Best Model:** 아무런 처리를 하지 않은 **Normal 상태의 Logistic Regression(0.279)**이 가장 높은 성능을 보입니다.
2. **Sampling 효과:** - **XGBoost**는 Borderline-SMOTE 적용 시 성능이 크게 개선되었습니다.
   - **LightGBM**은 Undersampling에서 가장 좋은 효율을 보여줍니다.
3. **종합 의견:** 데이터 불균형 처리가 항상 성능 향상을 보장하지는 않으므로, 현재 데이터셋에서는 모델별로 최적의 샘플링 방식이 다름을 알 수 있습니다.

In [ ]:
# from sklearn.metrics import (
#     roc_auc_score, average_precision_score,
#     confusion_matrix, classification_report
# )
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# best_name = "RandomForest"
# best_thr = 0.16

# model = models[best_name]
# model.fit(X_train, y_train)
# proba = model.predict_proba(X_test)[:, 1]
# pred = (proba >= best_thr).astype(int)

# print("ROC-AUC:", roc_auc_score(y_test, proba))
# print("PR-AUC :", average_precision_score(y_test, proba))

# # 퍼센트로 출력
# report = classification_report(y_test, pred, output_dict=True)
# df_report = pd.DataFrame(report).T
# for col in ["precision", "recall", "f1-score"]:
#     df_report[col] = (df_report[col] * 100).round(1)

# print(df_report[["precision","recall","f1-score","support"]])

# # Confusion Matrix 출력/시각화
# cm = confusion_matrix(y_test, pred)
# print("Confusion Matrix:\n", cm)

# plt.figure(figsize=(5, 4))
# sns.heatmap(
#     cm, annot=True, fmt="d", cmap="Greens",
#     xticklabels=["0", "1"], yticklabels=["0", "1"]
# )
# plt.title("Confusion Matrix")
# plt.xlabel("Predictions")
# plt.ylabel("Actuals")
# plt.tight_layout()
# plt.show()


In [ ]:
# import numpy as np
# from sklearn.metrics import f1_score, precision_score, recall_score, average_precision_score

# best_name = "RandomForest"
# model = models[best_name]

# model.fit(X_train, y_train)
# proba = model.predict_proba(X_valid)[:, 1]

# thresholds = np.linspace(0.01, 0.99, 99)
# best = {"thr": None, "f1": -1, "prec": 0, "rec": 0}

# for thr in thresholds:
#     pred = (proba >= thr).astype(int)
#     f1 = f1_score(y_valid, pred, zero_division=0)
#     if f1 > best["f1"]:
#         best["thr"] = thr
#         best["f1"] = f1
#         best["prec"] = precision_score(y_valid, pred, zero_division=0)
#         best["rec"] = recall_score(y_valid, pred, zero_division=0)

# print("Best threshold by F1:", best)
# print("PR-AUC (model only):", average_precision_score(y_valid, proba))
